# 🚀 国土交通省 地理空間情報データチャレンジ - Google Colab版

このノートブックは、ローカル環境のメモリ不足を回避するためのGoogle Colab実行用です。

## 📋 実行手順

1. **[STEP 1]** Google Driveをマウント
2. **[STEP 2]** GitHubリポジトリをクローン
3. **[STEP 3]** データファイルをアップロード（Google Driveから）
4. **[STEP 4]** 依存関係をインストール
5. **[STEP 5]** 地理空間特徴量ベースラインを実行
6. **[STEP 6]** 結果をダウンロード

## ⚠️ 注意事項

- Colab無料版: 12GB RAM（十分！）
- 実行時間: 約20-30分
- セッションタイムアウト: 12時間（アイドル時は90分）


## [STEP 1] Google Driveをマウント

データファイルとリポジトリをGoogle Driveに配置する場合は実行してください。


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Google Driveのマイドライブに移動
import os
os.chdir('/content/drive/MyDrive')
print("✅ Google Drive mounted successfully!")
print(f"📁 Current directory: {os.getcwd()}")


## [STEP 2] GitHubリポジトリをクローン

**オプション1**: 既にGoogle Driveにクローン済みの場合はスキップ  
**オプション2**: 新しくクローンする場合は実行


In [ ]:
# リポジトリをクローン（初回のみ）
!git clone https://github.com/TanimotoRui/mlit_geospatial_data_challenge_2025.git

# プロジェクトディレクトリに移動
%cd mlit_geospatial_data_challenge_2025

print("✅ Repository cloned!")
print(f"📁 Current directory: {os.getcwd()}")


## [STEP 3] データファイルを配置

以下のいずれかの方法でデータを配置してください：

### 方法1: Google Driveから手動でコピー（推奨）
1. Google Driveの`MyDrive`に`mlit_data`フォルダを作成
2. `train.csv`, `test.csv`, `sample_submit.csv`をアップロード
3. 下のセルを実行してコピー

### 方法2: Colabのファイルアップロード機能を使用
- 左サイドバーの📁アイコンから`data/raw/`にアップロード
- **注意**: セッション切れるとデータも消えます


In [ ]:
# 方法1: Google Driveからコピー（パスは適宜変更）
import shutil
from pathlib import Path

# data/raw ディレクトリを作成
Path("data/raw").mkdir(parents=True, exist_ok=True)

# Google Driveからコピー（パスは自分の環境に合わせて変更）
drive_data_path = Path("/content/drive/MyDrive/mlit_data")

if drive_data_path.exists():
    for file in ["train.csv", "test.csv", "sample_submit.csv"]:
        src = drive_data_path / file
        dst = Path("data/raw") / file
        if src.exists():
            shutil.copy(src, dst)
            print(f"✅ Copied: {file}")
        else:
            print(f"❌ Not found: {file}")
else:
    print(f"❌ Directory not found: {drive_data_path}")
    print("⚠️  Please create 'mlit_data' folder in MyDrive and upload data files")

# データファイルの確認
print("\n📁 Data files in data/raw/:")
!ls -lh data/raw/


## [STEP 4] 依存関係をインストール

必要なPythonパッケージをインストールします（約2-3分）。


In [ ]:
# 必要なパッケージをインストール
%pip install -q catboost scikit-learn pandas numpy pyarrow tqdm

print("✅ All packages installed!")

# バージョン確認
import catboost
import sklearn
import pandas as pd
import numpy as np

print(f"\n📦 Package versions:")
print(f"  - CatBoost: {catboost.__version__}")
print(f"  - scikit-learn: {sklearn.__version__}")
print(f"  - pandas: {pd.__version__}")
print(f"  - numpy: {np.__version__}")


## [STEP 5] 地理空間特徴量ベースラインを実行

**実行時間**: 約20-30分  
**メモリ**: 約8-10GB使用（Colab無料版で十分）

実行が完了すると、`submissions/exp003_geo_features/`に結果が保存されます。


In [ ]:
# スクリプトを実行
%run scripts/baseline_with_geo_features.py


## [STEP 6] 結果をダウンロード

実行が完了したら、submission ファイルをダウンロードします。


In [ ]:
# 結果ファイルを確認
!ls -lh submissions/exp003_geo_features/

# Google Colabからダウンロード
from google.colab import files
import glob

# 最新のsubmissionファイルを探す
submission_files = glob.glob("submissions/exp003_geo_features/submission_*.csv")
if submission_files:
    latest_submission = max(submission_files)
    print(f"📥 Downloading: {latest_submission}")
    files.download(latest_submission)
    print("✅ Download complete!")
else:
    print("❌ No submission file found")

# 特徴量重要度のプロットもダウンロード
importance_files = glob.glob("submissions/exp003_geo_features/feature_importance_*.png")
if importance_files:
    for file in importance_files:
        print(f"📥 Downloading: {file}")
        files.download(file)


## 📊 結果の確認

実行後、以下のファイルが生成されます：

- **submission_YYYYMMDD_HHMMSS.csv**: 提出用ファイル
- **feature_importance_fold_*.png**: 各Foldの特徴量重要度
- **feature_importance_average.png**: 平均特徴量重要度

## 💡 Tips

### メモリ不足が発生した場合

もし Colab でもメモリ不足が発生する場合は、パラメータを調整してください：

```python
# scripts/baseline_with_geo_features.py の以下を変更
n_clusters = 20  # 30 → 20
params["iterations"] = 200  # 300 → 200
params["depth"] = 3  # 4 → 3
n_splits = 2  # そのまま
```

### Colab Pro を使用する場合

- RAM: 25GB（無料版の2倍以上！）
- より高速なGPU（ただしCatBoostはCPUメインなので効果は限定的）
- 24時間セッション

### データの永続化

Google Driveにマウントしている場合、以下のディレクトリは永続化されます：
- `data/processed/`: 前処理済みデータ（2回目以降高速化）
- `submissions/`: 提出ファイル

## 🎯 次のステップ

1. ✅ このベースラインで提出
2. 🔧 パラメータチューニング（iterations, depth, learning_rate）
3. 🎨 新しい特徴量の追加
4. 🤖 アンサンブル（複数モデルの組み合わせ）

頑張ってください！🚀
